In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import matplotlib.pyplot as plt
import mplleaflet
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json

### The purpose of this notebook is to try to develop a bus trip planner using open data Brampton's geohub.

Note the naming convention for dataframes generated from files from the google_transit folder is prefixed with gt_ as in google_transit.

### Bus Stops
Each stop has an id, name, coordinates for latitude and longitude. We will drop all the other rows for simplicity.

In [4]:
gt_stops = pd.read_csv('google_transit/stops.txt', sep=",")
gt_stops = gt_stops.drop(['stop_code','stop_url', 'stop_desc', 'zone_id', 'stop_timezone', 'location_type', 'wheelchair_boarding', 'parent_station'], axis=1)

# This will treat all non numeric-looking strings as NaN
gt_stops['stop_id'] = gt_stops[gt_stops['stop_id'].str.isnumeric()==True]

# Drop all NaN rows
gt_stops.dropna(subset=['stop_id'], inplace=True)

# Cast numeric strings to int32
gt_stops['stop_id'] = gt_stops['stop_id'].astype('int32')

gt_stops.head()

,stop_id,stop_name,stop_lat,stop_lon
0,20,Kennedy Rd S n/of First Gulf Blvd,43.673256,-79.718468
1,30,Kennedy Rd S at Steeles Ave E,43.675159,-79.721710
2,55,Rutherford Rd S n/of Steeles Ave E,43.681385,-79.718147
3,60,Rutherford Rd S/of Bramsteele Rd,43.681969,-79.718979
4,70,Rutherford Rd S n/of Bramsteele Rd,43.683807,-79.721458


### Bus Routes
Each bus travels along a route, we want the id, and the route name. 

In [5]:
gt_routes = pd.read_csv('google_transit/routes.txt', sep=",")
gt_routes = gt_routes.drop(['route_url','route_desc', 'route_short_name','route_type'], axis=1) 
gt_routes.head()

,route_id,route_long_name
0,1-274,Queen
1,2-274,Main
2,3-274,McLaughlin
3,4-274,Chinguacousy
4,5-274,Bovaird


### Calendar 
Don't know if we will need this, though will keep it up for reference

In [6]:
gt_calendar = pd.read_csv('google_transit/calendar.txt', sep=",")
gt_calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,200302-MULTI-Weekday-01,1,1,1,1,1,0,0,20200302,20200424
1,200302-MULTI-Weekday-01-0001000,0,0,0,1,0,0,0,20200302,20200424
2,200302-MULTI-Weekday-01-0010000,0,0,1,0,0,0,0,20200302,20200424
3,200302-MULTI-Weekday-01-1000000,1,0,0,0,0,0,0,20200302,20200424
4,200302-MULTI-Saturday-01,0,0,0,0,0,1,0,20200307,20200425


### Stop Times
We are interested in the stop_id, and stop_sequence. They will be used later when making our adjacency list to represent a graph.

In [7]:
gt_stop_times = pd.read_csv('google_transit/stop_times.txt', sep=",")
gt_stop_times = gt_stop_times.drop(['timepoint', 'drop_off_type', 'pickup_type'], axis=1)
gt_stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,10445576-200302-MULTI-Weekday-01,05:10:00,05:10:00,45565,1
1,10445576-200302-MULTI-Weekday-01,05:12:00,05:12:00,5260,2
2,10445576-200302-MULTI-Weekday-01,05:13:00,05:13:00,5270,3
3,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5280,4
4,10445576-200302-MULTI-Weekday-01,05:14:00,05:14:00,5290,5


### Trips
The trip_id will be useful in connecting the above dataframe so that we can have both route_id and stop_id in a single dataframe. The direction column is either 0 or 1 referring to the two directions of the trip: towards the final destination, and back.

In [8]:
gt_trips = pd.read_csv('google_transit/trips.txt', sep=",")
gt_trips = gt_trips.drop(['wheelchair_accessible', 'bikes_allowed'], axis=1)
gt_trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
0,17-274,200302-MULTI-Weekday-01,10445576-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
1,17-274,200302-MULTI-Weekday-01,10445577-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
2,17-274,200302-MULTI-Weekday-01,10445578-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018
3,17-274,200302-MULTI-Weekday-01,10445579-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710328,170018
4,17-274,200302-MULTI-Weekday-01,10445580-200302-MULTI-Weekday-01,17 HOWDEN NORTH,0,710327,170018


We will merge select columns from the gt_trips and gt_stop_times dataframes to get a dataframe with route_id, trip_id, stop_id, along with the direction of travel and sequence of stops for each trip. Because trip_id is common to both dataframes, allowing us to use inner join.

In [9]:
routes_master = pd.merge(gt_trips[['route_id', 'trip_id', 'direction_id']], gt_stop_times[['trip_id', 'stop_id', 'stop_sequence']], how='inner')
routes_master.head()

,route_id,trip_id,direction_id,stop_id,stop_sequence
0,17-274,10445576-200302-MULTI-Weekday-01,0,45565,1
1,17-274,10445576-200302-MULTI-Weekday-01,0,5260,2
2,17-274,10445576-200302-MULTI-Weekday-01,0,5270,3
3,17-274,10445576-200302-MULTI-Weekday-01,0,5280,4
4,17-274,10445576-200302-MULTI-Weekday-01,0,5290,5


We will merge once again with the gt_stops dataframe to integrate the coordinates and names of each stop. Once done, we can drop the trip_id.

In [10]:
# This will be the primary dataframe
routes_master = pd.merge(routes_master, gt_stops[['stop_id', 'stop_name', 'stop_lon', 'stop_lat']], how='left')

routes_master.drop(columns=['trip_id'], inplace=True)
routes_master.drop_duplicates(inplace=True)

#routes_master.drop_duplicates(subset=['route_id', 'direction_id', 'stop_id', 'stop_sequence', 'stop_name'],inplace=True)
#routes_master.drop(columns=['trip_id'], inplace=True)

# Only use 1 direction
# routes_master[routes_master['direction_id'] == 0]
routes_master.head()

,route_id,direction_id,stop_id,stop_sequence,stop_name,stop_lon,stop_lat
0,17-274,0,45565,1,Bramalea Terminal Route 9 EB/17/207 Stop,-79.720367,43.718792
1,17-274,0,5260,2,Hanover Rd w/of Hartnell Sq,-79.724983,43.720787
2,17-274,0,5270,3,Hanover Rd at Helena Crt,-79.728081,43.718014
3,17-274,0,5280,4,Hanover Rd opp Hasting Square,-79.729599,43.718796
4,17-274,0,5290,5,Hanover Rd s/of Howden Blvd,-79.731422,43.720169


If we try querying via the route id 17-274, we can see the stops and names associated with direction 0

In [11]:
test = routes_master.loc[(routes_master['route_id']=='17-274') & (routes_master['direction_id']==0),:]
test

,route_id,direction_id,stop_id,stop_sequence,stop_name,stop_lon,stop_lat
0,17-274,0,45565,1,Bramalea Terminal Route 9 EB/17/207 Stop,-79.720367,43.718792
1,17-274,0,5260,2,Hanover Rd w/of Hartnell Sq,-79.724983,43.720787
2,17-274,0,5270,3,Hanover Rd at Helena Crt,-79.728081,43.718014
3,17-274,0,5280,4,Hanover Rd opp Hasting Square,-79.729599,43.718796
4,17-274,0,5290,5,Hanover Rd s/of Howden Blvd,-79.731422,43.720169
5,17-274,0,5300,6,Howden Blvd at Dixie Rd,-79.733772,43.719582
6,17-274,0,5310,7,Howden Blvd at Leander St,-79.737694,43.718353
7,17-274,0,5320,8,Howden Blvd at Vodden Rd E,-79.741463,43.717125
8,17-274,0,5330,9,Howden Blvd at Ladin Dr,-79.744026,43.718193
9,17-274,0,5340,10,Howden Blvd at Leander St,-79.745491,43.719364


## Create Dictionaries
Don't really know if these will be used. 

In [12]:
stop_dict = gt_stops.to_dict(orient='records')
stop_dict[:2]

[{'stop_id': 20,
  'stop_name': 'Kennedy Rd S n/of First Gulf Blvd',
  'stop_lat': 43.673256,
  'stop_lon': -79.718468},
 {'stop_id': 30,
  'stop_name': 'Kennedy Rd S at Steeles Ave E',
  'stop_lat': 43.675159,
  'stop_lon': -79.72171}]

In [13]:
route_dict = gt_routes.to_dict(orient='records')
route_dict[:2]

[{'route_id': '1-274', 'route_long_name': 'Queen'},
 {'route_id': '2-274', 'route_long_name': 'Main'}]

In [14]:
stop_times_dict = gt_stop_times.to_dict(orient='records')
stop_times_dict[:2]

[{'trip_id': '10445576-200302-MULTI-Weekday-01',
  'arrival_time': '05:10:00',
  'departure_time': '05:10:00',
  'stop_id': 45565,
  'stop_sequence': 1},
 {'trip_id': '10445576-200302-MULTI-Weekday-01',
  'arrival_time': '05:12:00',
  'departure_time': '05:12:00',
  'stop_id': 5260,
  'stop_sequence': 2}]

In [15]:
trips_dict = gt_trips.to_dict(orient='records')
trips_dict[:2]

[{'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445576-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710327,
  'shape_id': 170018},
 {'route_id': '17-274',
  'service_id': '200302-MULTI-Weekday-01',
  'trip_id': '10445577-200302-MULTI-Weekday-01',
  'trip_headsign': '17 HOWDEN NORTH',
  'direction_id': 0,
  'block_id': 710328,
  'shape_id': 170018}]

In [16]:
stop_desc_map = {stop['stop_name']:stop for stop in stop_dict}
stop_id_map = {stop['stop_id']:stop for stop in stop_dict} 

In [17]:
rm_dict = routes_master.to_dict(orient='records')
rm_dict[:3]

[{'route_id': '17-274',
  'direction_id': 0,
  'stop_id': 45565,
  'stop_sequence': 1,
  'stop_name': 'Bramalea Terminal Route 9 EB/17/207 Stop',
  'stop_lon': -79.72036700000001,
  'stop_lat': 43.718792},
 {'route_id': '17-274',
  'direction_id': 0,
  'stop_id': 5260,
  'stop_sequence': 2,
  'stop_name': 'Hanover Rd w/of Hartnell Sq',
  'stop_lon': -79.724983,
  'stop_lat': 43.720787},
 {'route_id': '17-274',
  'direction_id': 0,
  'stop_id': 5270,
  'stop_sequence': 3,
  'stop_name': 'Hanover Rd at Helena Crt',
  'stop_lon': -79.728081,
  'stop_lat': 43.718014000000004}]

### Plotting
Let's print route 17-274, direction 0 from the test dataset defined above

In [18]:
# Plot the routes
fig_routes, ax_routes = plt.subplots()

ax_routes.plot(test['stop_lon'],  test['stop_lat'], '.r')

# Display Inline
mplleaflet.display(fig=fig_routes)

# Graph Algorithms

We must first represent the graph as an adjacency list in the form of a dictionary of lists. 
In order to create the adjacency list, we can restructure the data to get a list of rows containing a node and the next node.
To do this, concatenate the column of stops in a route with the same column shifted up one cell

In [19]:
temp_edges = pd.concat([test['stop_id'], test['stop_id'].shift(-1)], axis=1)
temp_edges.columns = ['edge', 'next']
temp_edges.head()

,edge,next
0,45565,5260.0
1,5260,5270.0
2,5270,5280.0
3,5280,5290.0
4,5290,5300.0


In [70]:
# Adjacency List for stop_id
stop_graph = {}

def add_edge(row):
    edge, next_edge = row['edge'], row['next']
    
    # if edge key doesn't exist, create edge: [next]
    if edge not in stop_graph:
        stop_graph[int(edge)] = []
        
    # Check if next exists
        # if next does not exist, append to list
        # if next does exist, or is Nan dont do anything
    if next_edge not in stop_graph[edge] and not np.isnan(next_edge):
        stop_graph[int(edge)].append(int(next_edge))
        
        # Unidirectional connections between edges
        if next_edge not in stop_graph:
            stop_graph[int(next_edge)] = []
            
        if edge not in stop_graph[next_edge]:
            stop_graph[int(next_edge)].append(int(edge))
                
        
# iterate over each bus route and direction
unique_routes = routes_master['route_id'].unique()
for path in unique_routes:
    for direction in [0]:
        
        # get the dataframe
        p = routes_master.loc[(routes_master['route_id']==path) & (routes_master['direction_id']==direction),:]
        
        # create a dataframe containing edges and next edges
        #  because p shows sequence of bus stops, we shift the second column up by 1 unit 
        #  to get the following rows [Edge, Next_Edge] 
        edges = pd.concat([p['stop_id'], p['stop_id'].shift(-1)], axis=1) 
        edges.columns = ['edge', 'next']
            
        # The following adds edges to the graph
        edges.apply(add_edge, axis=1)
        
stop_graph

{45565: [5260, 3900, 23910, 18290, 25770],
 5260: [45565, 5270],
 5270: [5260, 5280],
 5280: [5270, 5290],
 5290: [5280, 5300],
 5300: [5290, 5310],
 5310: [5300, 5320],
 5320: [5310, 5330],
 5330: [5320, 5340],
 5340: [5330, 5350],
 5350: [5340, 5360],
 5360: [5350, 5370],
 5370: [5360, 5380],
 5380: [5370, 5390],
 5390: [5380, 5400],
 5400: [5390, 5410],
 5410: [5400, 16570],
 16570: [5410, 14090136],
 14090136: [16570],
 20: [30, 440, 19575],
 30: [20, 51100, 19610, 42400],
 51100: [30, 55, 620, 440, 15021601, 30109, 51102],
 55: [51100, 60],
 60: [55, 70],
 70: [60, 80],
 80: [70, 90],
 90: [80, 100],
 100: [90, 110],
 110: [100, 120],
 120: [110, 130],
 130: [120, 140],
 140: [130, 150],
 150: [140, 160],
 160: [150, 170],
 170: [160, 180],
 180: [170, 190],
 190: [180, 200],
 200: [190, 210, 7800],
 210: [200, 220],
 220: [210, 230],
 230: [220, 235, 2720],
 235: [230, 9875],
 9875: [235, 9885],
 9885: [9875, 2410, 23140, 10360, 45570],
 2410: [9885, 23140, 3865],
 23140: [2410, 

#### Depth-First Search

In [50]:
def dfs(graph, start, end):
    stack = []
    visited = []
    
    # Add the first value to the queue manually to allow 
    # the while loop to execute
    stack.append(start)
    
    while(stack):
        
        # pop off the last element in the stack
        curr = stack.pop(-1)
        
        # Add it to the visited list
        visited.append(curr)
        
        # If we have reached our target node, then exit
        if curr == end:
            return visited
        
        # The order of iteration doesn't change much,
        #  reversed means that we travel down the leftmost branch first
        for next_node in reversed(graph[curr]):
            if next_node not in visited:
                stack.append(next_node)
            
    # Return start path if not found  
    return [start]

route = []   

route = dfs(stop_graph, 45565, 5350)

for stop in route:
    stop_name = gt_stops[gt_stops['stop_id']==stop]['stop_name'].values[0]
    print(f'stop_id: {stop:<15} stop_name: {stop_name}')

stop_id: 45565           stop_name: Bramalea Terminal Route 9 EB/17/207 Stop
stop_id: 5260            stop_name: Hanover Rd w/of Hartnell Sq
stop_id: 5270            stop_name: Hanover Rd at Helena Crt
stop_id: 5280            stop_name: Hanover Rd opp Hasting Square
stop_id: 5290            stop_name: Hanover Rd s/of Howden Blvd
stop_id: 5300            stop_name: Howden Blvd at Dixie Rd
stop_id: 5310            stop_name: Howden Blvd at Leander St
stop_id: 5320            stop_name: Howden Blvd at Vodden Rd E
stop_id: 5330            stop_name: Howden Blvd at Ladin Dr
stop_id: 5340            stop_name: Howden Blvd at Leander St
stop_id: 5350            stop_name: North Park Dr n/of Williams Pwky E


#### Breadth-First Search


In [60]:
def bfs(graph, start, end):
    queue = []
    visited = []
    
    # Add the first value to the queue manually to allow 
    # the while loop to execute
    queue.append(start)
    
    while(queue):
        
        # pop off the first element in the queue
        curr = queue.pop(0)
        # Add it to the visited list
        visited.append(curr)
        
        # If we have reached our target node, then exit
        if curr == end:
            return visited
        
        # Add the undiscovered nodes connected to the current node to the queue
        for next_node in graph[curr]:
            if next_node not in visited:
                queue.append(next_node)
    
    # Return start path if not found  
    return [start]
    
route = []

route = bfs(stop_graph, 45565, 5350)

for stop in route:
    stop_name = gt_stops[gt_stops['stop_id']==stop]['stop_name'].values[0]
    print(f'stop_id: {stop:<15} stop_name: {stop_name}')

stop_id: 45565           stop_name: Bramalea Terminal Route 9 EB/17/207 Stop
stop_id: 5260            stop_name: Hanover Rd w/of Hartnell Sq
stop_id: 3900            stop_name: Knightsbridge Rd btwn Central Park & Kings Cross
stop_id: 18290           stop_name: Central Park Dr n/of Hanover Rd
stop_id: 5270            stop_name: Hanover Rd at Helena Crt
stop_id: 3910            stop_name: Knightsbridge Rd at Bramalea Rd
stop_id: 18300           stop_name: Central Park Dr n/of Howden Blvd
stop_id: 25480           stop_name: Howden Blvd at Hunt Club Dr
stop_id: 5280            stop_name: Hanover Rd opp Hasting Square
stop_id: 20200           stop_name: Bramalea Rd n/of Fleetwood Cres
stop_id: 4400            stop_name: Bramalea Rd s/of Clark Blvd
stop_id: 9890            stop_name: Clark Blvd at Edenborough Dr
stop_id: 18310           stop_name: Central Park Dr e/of Hilldale Cres (W)
stop_id: 6740            stop_name: Hilldale Cres at Hayden Crt
stop_id: 25490           stop_name: Howden

In [62]:
# Just as a sanity check, lets see the outputs of both algorithms on a simple tree
test_graph = {1: [2, 3],
              2: [4, 5],
              3: [6, 7],
              4: [8],
              5: [],
              6: [],
              7: [],
              8: []}

route_dfs = dfs(test_graph, 1, 7)
route_bfs = bfs(test_graph, 1, 7)

print(route_dfs)
print(route_bfs)

[1, 2, 4, 8, 5, 3, 6, 7]
[1, 2, 3, 4, 5, 6, 7]


Next, we want to factor in the distances between two nodes. This is represented as a weighted graph. An assumption (not always true, but better than nothing) will be that the path between bus stops is a straight line. As we have access to bus stop locations via latitude and longitude coordinates, we can use the [haversine](https://www.movable-type.co.uk/scripts/latlong.html) formula to calcualate the great-circle distance between two points on a sphere (basically the arc length).  

In [63]:
# Haversine distance between two coordinates in metres
def get_dist_h(stop_a, stop_b):
    R = 6371*1000 # Earth's radius
    
    # Get lat in radians
    stop_a_lat = gt_stops[gt_stops['stop_id'] == stop_a]['stop_lat'].values[0] 
    stop_b_lat = gt_stops[gt_stops['stop_id'] == stop_b]['stop_lat'].values[0]
    
    # Get lon in radians
    stop_a_lon = gt_stops[gt_stops['stop_id'] == stop_a]['stop_lon'].values[0] 
    stop_b_lon = gt_stops[gt_stops['stop_id'] == stop_b]['stop_lon'].values[0] 
    
    Phi_1 = stop_a_lat * np.pi / 180.0
    Phi_2 = stop_b_lat * np.pi / 180.0
    
    Phi_delta = (stop_b_lat - stop_a_lat) * np.pi / 180.0
    Lambda_delta = (stop_b_lon - stop_a_lon) * np.pi / 180.0
    
    
    delta_lat = (stop_b_lat - stop_a_lat) * np.pi / 180.0 
    delta_lon = (stop_b_lon - stop_a_lon) * np.pi / 180.0 
    
    a = np.sin(Phi_delta/2)**2 + np.cos(Phi_1) * np.cos(Phi_2) * np.sin(Lambda_delta/2)**2
    
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return R*c

a = 5320
b = 5330
print(f'distance between stops {a} and {b} is {get_dist_h(a, b)} metres')

distance between stops 5320 and 5330 is 237.7616917050869 metres


In [64]:
class Graph:
    
    def __init__(self):
        self.adj_list = {}

    def add_edge(self, row):
        edge, next_edge = row['edge'], row['next']
        
        # if edge key doesn't exist, create edge: [next]
        if edge not in self.adj_list:
            self.adj_list[int(edge)] = {}

        # Check if next exists
            # if next does not exist, append to list
            # if next does exist, or is Nan dont do anything
        if next_edge not in self.adj_list[edge] and not np.isnan(next_edge):
            self.adj_list[edge][int(next_edge)] = get_dist_h(edge, next_edge)
    
    
g = Graph()

# iterate over each bus route and direction
unique_routes = routes_master['route_id'].unique()
for path in unique_routes:
    for direction in [0,1]:
        
        # get the dataframe
        p = routes_master.loc[(routes_master['route_id']==path) & (routes_master['direction_id']==direction),:]
        
        # create a dataframe containing edges and next edges
        #  because p shows sequence of bus stops, we shift the second column up by 1 unit 
        #  to get the following rows [Edge, Next_Edge] 
        edges = pd.concat([p['stop_id'], p['stop_id'].shift(-1)], axis=1) 
        edges.columns = ['edge', 'next']
        
        # The following adds edges to the graph
        edges.apply(lambda x: g.add_edge(x), axis=1)
        
g.adj_list

{45565: {5260: 432.22779545655845,
  3900: 362.5710332319548,
  18290: 489.2498984270058},
 5260: {5270: 396.3092718923764},
 5270: {5280: 149.8127084033746},
 5280: {5290: 211.59324250412618},
 5290: {5300: 199.8158840088842},
 5300: {5310: 343.54166370779467},
 5310: {5320: 332.2565017548784, 25530: 408.478955464326},
 5320: {5330: 237.7616917050869, 25530: 97.57756365022085},
 5330: {5340: 175.54472298812374},
 5340: {5350: 201.3169137346362},
 5350: {5360: 394.8371602778574},
 5360: {5370: 219.18814512418652},
 5370: {5380: 362.95732075361525},
 5380: {5390: 247.3464952657706},
 5390: {5400: 117.36443862177153},
 5400: {5410: 170.04326911814303},
 5410: {16570: 172.58586239674125},
 16570: {14090136: 263.72337816158495},
 14090136: {11825: 235.84751885123094, 14090134: 1151.7007403140378},
 11825: {5450: 537.1491900009202, 11060: 164.82158698121373},
 5450: {5460: 249.54851901670887},
 5460: {5470: 285.8651671109938},
 5470: {5480: 277.1265654384374},
 5480: {5490: 315.178481152759

In [65]:
# Adjacency List for stop_id
stop_graph = {}        

def add_edge(row):
    edge, next_edge = row['edge'], row['next']
    
    # if edge key doesn't exist, create edge: [next]
    if edge not in stop_graph:
        stop_graph[int(edge)] = {}
    
    # Check if next exists
        # if next does not exist, append to list
        # if next does exist, or is Nan dont do anything
    if next_edge not in stop_graph[edge] and not np.isnan(next_edge):
        stop_graph[edge][int(next_edge)] = get_dist_h(edge, next_edge)
        
# iterate over each bus route and direction
unique_routes = routes_master['route_id'].unique()
for path in unique_routes:
    for direction in [0,1]:
        
        # get the dataframe
        p = routes_master.loc[(routes_master['route_id']==path) & (routes_master['direction_id']==direction),:]
        
        # create a dataframe containing edges and next edges
        #  because p shows sequence of bus stops, we shift the second column up by 1 unit 
        #  to get the following rows [Edge, Next_Edge] 
        edges = pd.concat([p['stop_id'], p['stop_id'].shift(-1)], axis=1) 
        edges.columns = ['edge', 'next']
        
        # The following adds edges to the graph
        edges.apply(add_edge, axis=1)


In [66]:
from queue import PriorityQueue
import sys
import numpy as np

from heapq import heapify, heappush, heappop 


# Note that this is an undirected graph
test_graph = test_graph = {10: {20:1, 30:3, 80:2},
              20: {40:4, 50:7},
              30: {60:2, 70:1},
              40: {80:3},
              50: {80:1},
              60: {},
              70: {},
              80: {}}


def dijkstra_paths(graph, start):
    length = len(graph)
    
    nodes = np.array(list(graph.keys()))
    
    dist = np.ones(length) * float('inf')
    loc = np.where(nodes == start)[0][0] # Returns tuple with array of matches and dtype
    dist[loc] = 0
    
    Q = []
    
    for i in range(length):
        Q.append((dist[i], nodes[i]))
    
    heapify(Q)
    
    parent = [-1] * length
    
    while Q:
        curr = Q.pop(0)
        
        for neighbour in graph[curr[1]]:
            loc_neighbour = np.where(nodes == neighbour)[0][0]
            loc_curr = np.where(nodes == curr[1])[0][0]
            
            if dist[loc_neighbour] > dist[loc_curr] + graph[curr[1]][neighbour]:                
                dist[loc_neighbour] = float(dist[loc_curr]) + graph[curr[1]][neighbour]
                parent[loc_neighbour] = loc_neighbour
    return dist, nodes, parent
        
    
d, n, p = dijkstra_paths(test_graph, 10)
print(d)
print(n)
print(p)

[0. 1. 3. 5. 8. 5. 4. 2.]
[10 20 30 40 50 60 70 80]
[-1, 1, 2, 3, 4, 5, 6, 7]


In [67]:
d, n, p = dijkstra_paths(stop_graph, 4520)

In [68]:
print(stop_graph[4520])

print(stop_graph[3500])

{4530: 213.6514292787627}
{3510: 282.6104622868537}


In [69]:
a = 4520
b = 4530
print(f'Distance from {a} to {b} is ', d[np.where(n == b)[0][0]])

Distance from 4520 to 4530 is  213.6514292787627


#### References
- https://www.lihaoyi.com/post/PlanningBusTripswithPythonSingaporesSmartNationAPIs.html#breadth-first-search
- https://geohub.brampton.ca/pages/brampton-transit 
- http://zetcode.com/python/fstring/
- https://www.movable-type.co.uk/scripts/latlong.html